In [1]:
import aocd
import numpy as np
from copy import copy
data = list([int(d) for d in aocd.get_data(day=19).split(",")])
print(data)

[109, 424, 203, 1, 21102, 11, 1, 0, 1105, 1, 282, 21101, 0, 18, 0, 1105, 1, 259, 2101, 0, 1, 221, 203, 1, 21102, 1, 31, 0, 1105, 1, 282, 21102, 1, 38, 0, 1105, 1, 259, 20102, 1, 23, 2, 21201, 1, 0, 3, 21102, 1, 1, 1, 21102, 57, 1, 0, 1106, 0, 303, 2102, 1, 1, 222, 21002, 221, 1, 3, 20101, 0, 221, 2, 21101, 0, 259, 1, 21101, 0, 80, 0, 1105, 1, 225, 21101, 44, 0, 2, 21102, 91, 1, 0, 1105, 1, 303, 1202, 1, 1, 223, 21002, 222, 1, 4, 21102, 259, 1, 3, 21102, 1, 225, 2, 21102, 225, 1, 1, 21101, 118, 0, 0, 1106, 0, 225, 21002, 222, 1, 3, 21101, 163, 0, 2, 21101, 0, 133, 0, 1106, 0, 303, 21202, 1, -1, 1, 22001, 223, 1, 1, 21102, 148, 1, 0, 1106, 0, 259, 1202, 1, 1, 223, 20101, 0, 221, 4, 21001, 222, 0, 3, 21102, 1, 24, 2, 1001, 132, -2, 224, 1002, 224, 2, 224, 1001, 224, 3, 224, 1002, 132, -1, 132, 1, 224, 132, 224, 21001, 224, 1, 1, 21101, 195, 0, 0, 105, 1, 108, 20207, 1, 223, 2, 21002, 23, 1, 1, 21102, -1, 1, 3, 21102, 1, 214, 0, 1106, 0, 303, 22101, 1, 1, 1, 204, 1, 99, 0, 0, 0, 0, 109, 5,

In [2]:
IN_SIZE = {1: 4, 2: 4, 3: 2, 4: 2, 5: 3, 6: 3, 7: 4, 8: 4, 9: 2, 99: 1}
IN_NAME =  {1: "add", 2: "mul", 3: "rd", 4: "prnt", 5: "jnz", 6: "jz", 
            7: "lt", 8: "eq", 9: "bas", 99: "ret"}
EXT_MEM = 100

class Process():  # wrapper for generator
    def __init__(self, data, ptr=0, dbg=False): 
        self.d = data[:]+[0]*EXT_MEM  # copy + extend memory
        self.done = False
        self.base = 0
        self.ptr = ptr
    
    def parse_ins(self, ptr, dbg=False):
        param = [0, 0, 0]
        ins = self.d[ptr]%100
        modes = [self.d[ptr]//10**e%10 for e in range(2,5)]
        for i, mode in enumerate(modes):
            size = IN_SIZE[ins]-1
            if i < size:
                p = ptr+1+i
                if mode == 0:  param[i] = self.d[p]          # position
                if mode == 1:  param[i] = p                  # intermediate
                if mode == 2:  param[i] = self.base+self.d[p]# relative
        if dbg: print(ptr, IN_NAME[ins], param[:size], 
                      self.d[ptr:ptr+4], sep = "\t")# debug print
        return [ins] + param

    def process(self, inp, dbg=False): 
        out = []; ptr = self.ptr; d = self.d  # initializations
        parse = lambda i: [i%100] + [i//10**e%10 for e in range(2,5)]
        while ptr < len(d):                                     # stop on EOF
            ins, p1, p2, p3 = self.parse_ins(ptr, dbg=dbg)
            # if dbg:print(ptr, d)                              # debug print
            if   ins == 1: d[p3] = d[p1] + d[p2]                # add
            elif ins == 2: d[p3] = d[p1] * d[p2]                # mul
            elif ins == 3:                                      # read
                if not inp: self.ptr = ptr; return out;         # wait/flush
                d[p1] = inp.pop(0)                              # read
            elif ins == 4: out.append(d[p1])                    # print
            elif ins == 5: ptr = d[p2]-3 if     d[p1] else ptr  # jnz
            elif ins == 6: ptr = d[p2]-3 if not d[p1] else ptr  # jz
            elif ins == 7: d[p3] = int(d[p1] < d[p2])           # lt
            elif ins == 8: d[p3] = int(d[p1] == d[p2])          # eq
            elif ins == 9: self.base += d[p1]                   # base
            elif ins ==99: self.done=True; return out           # ret
            else: print(f"invalid opcode {ins} @ {ptr}")        # err
            ptr += IN_SIZE[ins] # jmp is compensated with -3    # move ptr

# tests in other files

In [3]:
def init(x, y):
    a = np.zeros((x,y), dtype=int)
    for pos, val in np.ndenumerate(a):
        a[pos]=Process(data).process(list(pos))[0]
    return a

aocd.submit(np.sum(init(50,50)), day=19)

answer a: 179
submitting for part b (part a is already completed)


aocd will not submit that answer again. You've previously guessed 179 and the server responded:
That's not the right answer; your answer is too low.  If you're stuck, make sure you're using the full input data; there are also some general tips on the about page, or you can ask for hints on the subreddit.  Please wait one minute before trying again. (You guessed 179.) [Return to Day 19]


In [4]:
score = lambda p: p[0]*10000+p[1]
def find_square(a, size):
    a = a.copy()  # do not modify the original!
    res = [(x,y) for (x,y), _ in np.ndenumerate(a) if np.sum(a[x:x+size, y:y+size]) == size*size]
    return min(res) if res else []

a = init(1100, 600)  # 10/10 works with 110/60 so we use x10 here
aocd.submit(score(find_square(a, 100)), day=19)

answer a: 179
submitting for part b (part a is already completed)
posting 9760485 to https://adventofcode.com/2019/day/19/answer (part b) token=...2749


You don't seem to be solving the right level.  Did you already complete it? [Return to Day 19]


<Response [200]>